# Test MCP Date Tool Integration

## Purpose

This notebook demonstrates how the Needle and Summary agents use the MCP Date Difference tool for deterministic date calculations.

## Why MCP Tools?

### ❌ Why NOT Use LLM for Date Calculations:
- LLMs **approximate** ("about 25 days")
- LLMs **forget** leap years and month boundaries
- LLMs **hallucinate** numbers
- No guarantee of correctness

### ✅ Why Use MCP Tool:
- **Exact** calculation (25 days, not "about 25")
- **Deterministic** (same input = same output)
- **Verifiable** (can check the math)
- **Handles edge cases** (leap years, month boundaries)

## How It Works

```
User Query: "How many days between Jon Mor's accident and repair?"
        ↓
Agent retrieves context:
    "Accident Date: 2024-01-24
     Repair Appointment: 2024-02-18"
        ↓
LLM sees dates in context
LLM recognizes: "Need to calculate days!"
        ↓
LLM calls: calculate_days_between('2024-01-24', '2024-02-18')
        ↓
MCP Tool returns: {"number_of_days": 25}
        ↓
LLM formats: "25 days passed between..."
```

**Key Point:** The LLM decides when to use the tool. No Python conditionals!

## Setup

In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv

# Setup paths
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")
print(f"✅ Environment loaded")

✅ Project root: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2
✅ Environment loaded


In [2]:
# Import agents and index manager
from RAG.Agents.needle_agent import create_needle_agent
from RAG.Agents.summary_agent import create_summary_agent
from RAG.Index_Layer.index_layer import ClaimIndexManager

print("✅ Imports successful")

✅ Imports successful


## Load Production Index

In [3]:
# Load the production index
print("Loading production index...")
index_manager = ClaimIndexManager()
index_manager.load_index(persist_dir=str(project_root / "production_index"))
print("✅ Index loaded successfully")

Loading production index...
📂 Loading index from: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2/production_index
✅ Created embedding model: text-embedding-3-small
   This is the SINGLE embedding instance for all operations
✅ Loaded indexes from: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2/production_index
   Claim #ALL_20_CLAIMS
   Embedding: text-embedding-3-small
✅ Index loaded and ready
✅ Index loaded successfully


## Test 1: Needle Agent with Date Calculation

**Scenario:** Jon Mor's accident and repair dates

**Query:** "How many days passed between Jon Mor's accident and his repair appointment?"

**Expected:**
1. Agent retrieves context about Jon Mor
2. Context contains: Accident (2024-01-24), Repair (2024-02-18)
3. LLM recognizes date calculation needed
4. LLM calls MCP tool with extracted dates
5. Tool returns: 25 days
6. LLM formats answer

In [4]:
# Create Needle Agent with MCP tools enabled
needle_agent = create_needle_agent(
    model="gpt-4o-mini",
    temperature=0.0,
    enable_mcp_tools=True  # MCP tools enabled!
)

✅ Needle Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.0
   Output: Structured (Pydantic)
   Policy: NO GUESSING
   MCP Tools: ✅ Enabled


In [5]:
# Create retriever for needle queries
needle_retriever = index_manager.get_needle_retriever(
    top_k=5,
    similarity_threshold=0.7
)

print("✅ Retriever created")

🎯 Needle Retriever configured:
   Scope: Child chunks only
   Top-k: 5
   Similarity threshold: 0.7
   Uses embedding: text-embedding-3-small
✅ Retriever created


In [6]:
# Test query requiring date calculation
question = "How many days passed between Jon Mor's accident and his repair appointment?"

print(f"\n💬 Question: {question}")
print("\n🔍 Processing...\n")

result = needle_agent.answer(question, needle_retriever)

print("\n" + "="*70)
print("📊 RESULT")
print("="*70)
print(f"Answer: {result['answer']}")
print(f"Confidence: {result['confidence']}")
print(f"Sources: {len(result['sources'])} chunk(s)")
print(f"Reason: {result['reason']}")
print("="*70)

# Check if MCP tool was used
if "MCP" in result['reason']:
    print("\n✅ SUCCESS: MCP tool was called for date calculation!")
else:
    print("\n⚠️  Note: MCP tool was not called (may not have been needed)")


💬 Question: How many days passed between Jon Mor's accident and his repair appointment?

🔍 Processing...


🔍 Retrieving chunks for: 'How many days passed between Jon Mor's accident and his repair appointment?'
   ✅ Retrieved 5 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...
   🔧 MCP tools enabled - LLM can use tools if needed
   🔧 LLM called MCP tool for date calculation
      Calculating: 2024-06-06 to 2024-07-08
      Result: 32 days
   🤖 LLM formatting answer with tool result...

📊 RESULT
Answer: 32 days
Confidence: 0.95
Sources: 1 chunk(s)
Reason: Calculated the number of days between the accident on 2024-06-06 and the repair appointment on 2024-07-08. (Used MCP: calculate_days_between)

✅ SUCCESS: MCP tool was called for date calculation!


## Test 2: Summary Agent with Timeline

**Scenario:** Timeline synthesis with date duration

**Query:** "Describe Jon Mor's claim timeline, including how long it took from accident to repair"

**Expected:**
1. Agent retrieves comprehensive context
2. LLM synthesizes timeline narrative
3. LLM recognizes date calculation needed
4. LLM calls MCP tool for exact duration
5. LLM integrates "25 days" into narrative

In [7]:
# Create Summary Agent with MCP tools enabled
summary_agent = create_summary_agent(
    model="gpt-4o-mini",
    temperature=0.2,
    enable_mcp_tools=True  # MCP tools enabled!
)

✅ Summary Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.2
   Output: Structured (Pydantic)
   Policy: CONTEXT-GROUNDED SYNTHESIS
   MCP Tools: ✅ Enabled


In [8]:
# Create retriever for summary queries
summary_retriever = index_manager.get_summary_retriever(top_k=10)

print("✅ Summary retriever created")

📚 Summary Retriever configured:
   Scope: Parent + Child chunks
   Top-k: 10
   Similarity threshold: None (high recall)
   Uses embedding: text-embedding-3-small
✅ Summary retriever created


In [9]:
# Test query requiring timeline with date calculation
question = "Describe Jon Mor's claim timeline, including how long it took from accident to repair appointment"

print(f"\n💬 Question: {question}")
print("\n🔍 Processing...\n")

result = summary_agent.answer(question, retriever=summary_retriever)

print("\n" + "="*70)
print("📊 RESULT")
print("="*70)
print(f"Answer:\n{result['answer']}\n")
print(f"Confidence: {result['confidence']}")
print(f"Sources: {len(result['sources'])} chunk(s)")
print(f"Reason: {result['reason']}")
print("="*70)

# Check if MCP tool was used
if "MCP" in result['reason']:
    print("\n✅ SUCCESS: MCP tool was called for date calculation!")
else:
    print("\n⚠️  Note: MCP tool was not called (may not have been needed)")


💬 Question: Describe Jon Mor's claim timeline, including how long it took from accident to repair appointment

🔍 Processing...


📚 Retrieving context for: 'Describe Jon Mor's claim timeline, including how long it took from accident to repair appointment'
   ✅ Retrieved 10 chunk(s)
   🤖 Synthesizing answer with gpt-4o-mini...
   🔧 MCP tools enabled - LLM can use tools if needed
   🔧 LLM called MCP tool for date calculation
      Calculating: 2024-06-06 to 2024-07-08
      Result: 32 days
   🤖 LLM formatting answer with tool result...

📊 RESULT
Answer:
Jon Mor's claim timeline begins with the accident that occurred on June 6, 2024, at the intersection of 10th Ave and 5th St in Sample City. The accident was a rear-end collision, which resulted in minor injuries and moderate damage to his vehicle, a 2015 Toyota Corolla. Following the accident, a police report was filed, and the claim status is currently pending court. The repair appointment for the vehicle is scheduled for July 8, 2024, w

## Test 3: Regular Query (No Date Calculation)

**Purpose:** Verify that tools don't interfere with regular queries

**Query:** "What is Jon Mor's phone number?"

**Expected:**
- Agent answers normally
- NO MCP tool call
- Direct fact extraction

In [10]:
# Test regular query (no calculation needed)
question = "What is Jon Mor's phone number?"

print(f"\n💬 Question: {question}")
print("\n🔍 Processing...\n")

result = needle_agent.answer(question, needle_retriever)

print("\n" + "="*70)
print("📊 RESULT")
print("="*70)
print(f"Answer: {result['answer']}")
print(f"Confidence: {result['confidence']}")
print(f"Tool Used: {'YES' if 'MCP' in result['reason'] else 'NO'}")
print("="*70)

# Verify no tool was called
if "MCP" not in result['reason']:
    print("\n✅ CORRECT: No MCP tool call for regular fact extraction")
else:
    print("\n⚠️  Unexpected: Tool was called when not needed")


💬 Question: What is Jon Mor's phone number?

🔍 Processing...


🔍 Retrieving chunks for: 'What is Jon Mor's phone number?'
   ✅ Retrieved 5 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...
   🔧 MCP tools enabled - LLM can use tools if needed

📊 RESULT
Answer: (555) 100-2000
Confidence: 0.95
Tool Used: NO

✅ CORRECT: No MCP tool call for regular fact extraction


## Summary

### What We Demonstrated:

1. **Date Calculation (Needle Agent)**
   - Query asks about time between dates
   - LLM automatically calls MCP tool
   - Returns exact number: **25 days**

2. **Timeline Synthesis (Summary Agent)**
   - Query asks for timeline with duration
   - LLM calls MCP tool for calculation
   - Integrates result into narrative

3. **Regular Query (No Tools)**
   - Simple fact extraction
   - No tool call needed
   - Works normally

### Key Insights:

✅ **LLM Decides** - No Python conditionals, LLM chooses when to use tools  
✅ **Exact Results** - No "approximately" or "about"  
✅ **Seamless** - Regular queries work exactly the same  
✅ **Verifiable** - Can check the date arithmetic externally  

### Why This is Better Than Prompting:

**Without MCP Tool:**
```
LLM: "Approximately 24 or 25 days passed..."
     ❌ Imprecise
     ❌ Might be wrong
     ❌ Cannot verify
```

**With MCP Tool:**
```
Tool: {"number_of_days": 25}
LLM: "Exactly 25 days passed..."
     ✅ Precise
     ✅ Guaranteed correct
     ✅ Verifiable
```

### Next Steps:

1. ✅ Both agents support MCP tools
2. ✅ LLM decides when to use them
3. ✅ Backward compatible with existing queries
4. 🚀 Ready for production use!

To disable MCP tools:
```python
needle_agent = create_needle_agent(enable_mcp_tools=False)
summary_agent = create_summary_agent(enable_mcp_tools=False)
```